In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
from PIL import Image
import pydicom
from sklearn.model_selection import train_test_split
import nibabel as nib
import seaborn as sns
from tqdm import tqdm

import pytorch_lightning as L
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class Config:
    SEED = 101
    BATCH_SIZE = 16
    MAX_EPOCHS = 5
    LR=0.005

config = Config()
random.seed(config.SEED)
np.random.seed(config.SEED)

In [3]:
original = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
test_or = pd.read_parquet('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_dicom_tags.parquet')

In [4]:
df_injuried = original[original['any_injury']==1]
injuried = df_injuried['patient_id']

In [29]:
df_not_injuried = original[original['any_injury']==0]
not_injuried = df_not_injuried['patient_id']

In [30]:
len(injuried)

855

In [31]:
len(not_injuried)

2292

In [32]:
df_not_injuried_200 = df_not_injuried[:200]
not_injuried_200 = df_not_injuried_200['patient_id']

In [33]:
len(df_not_injuried_200)

200

In [39]:
#Creation of the path of the DICOM images
root_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
patients = injuried

serial_path = {'patient_id':[],
               'serial_id':[],
               'serial_path':[]
              }

serials = {'patient_path':[],
           'patient': []
          }
# Populate serials['patient_path'] first
for patient_id in patients:
    patient_path = os.path.join(root_dir, str(patient_id))
    serials['patient_path'].append(patient_path)
    serials['patient'].append(patient_id)
    
serial_list = list(serials.values())
patient_paths, patient_ids = serial_list

#Now, let's create entries in serial_path for each serial ID
for patient_path, patient_id in zip(patient_paths, patient_ids):
    serial_ids = os.listdir(patient_path)
    for serial_id in serial_ids:
        serial_path['patient_id'].append(patient_id)
        serial_path['serial_id'].append(serial_id)
        serial_path['serial_path'].append(os.path.join(patient_path,serial_id))

df_serials_inj = pd.DataFrame(serial_path)

image_paths = {'patient_id':[],'serial_id':[],'serial_path':[],'image_path':[], 'image':[] }

patient_path_dir = df_serials_inj['patient_id']
serial_id_dir = df_serials_inj['serial_id']
serial_path_dir = df_serials_inj['serial_path']

for patient_id, serial_id, serial_path in zip(patient_path_dir, serial_id_dir, serial_path_dir):
    images = os.listdir(serial_path)
    for image in images:
        image_paths['patient_id'].append(patient_id)
        image_paths['serial_id'].append(serial_id)
        image_paths['image_path'].append(os.path.join(serial_path,image))
        image_paths['serial_path'].append(serial_path)
        image_paths['image'].append(image)
        
image_paths_df = pd.DataFrame(image_paths)

image_paths_df['image'] = image_paths_df['image'].str.replace('.dcm', '').astype(int)

In [40]:
#Creation of the path of the DICOM images
root_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
patients = not_injuried_200

serial_path = {'patient_id':[],
               'serial_id':[],
               'serial_path':[]
              }

serials = {'patient_path':[],
           'patient': []
          }
# Populate serials['patient_path'] first
for patient_id in patients:
    patient_path = os.path.join(root_dir, str(patient_id))
    serials['patient_path'].append(patient_path)
    serials['patient'].append(patient_id)
    
serial_list = list(serials.values())
patient_paths, patient_ids = serial_list

#Now, let's create entries in serial_path for each serial ID
for patient_path, patient_id in zip(patient_paths, patient_ids):
    serial_ids = os.listdir(patient_path)
    for serial_id in serial_ids:
        serial_path['patient_id'].append(patient_id)
        serial_path['serial_id'].append(serial_id)
        serial_path['serial_path'].append(os.path.join(patient_path,serial_id))

df_serials_n_inj = pd.DataFrame(serial_path)

image_paths = {'patient_id':[],'serial_id':[],'serial_path':[],'image_path':[], 'image':[] }

patient_path_dir = df_serials_n_inj['patient_id']
serial_id_dir = df_serials_n_inj['serial_id']
serial_path_dir = df_serials_n_inj['serial_path']

for patient_id, serial_id, serial_path in zip(patient_path_dir, serial_id_dir, serial_path_dir):
    images = os.listdir(serial_path)
    for image in images:
        image_paths['patient_id'].append(patient_id)
        image_paths['serial_id'].append(serial_id)
        image_paths['image_path'].append(os.path.join(serial_path,image))
        image_paths['serial_path'].append(serial_path)
        image_paths['image'].append(image)
        
image_paths_df_not_inj = pd.DataFrame(image_paths)

image_paths_df_not_inj['image'] = image_paths_df_not_inj['image'].str.replace('.dcm', '').astype(int)

In [20]:
len(image_paths_df_not_inj)

89194

In [21]:
len(image_paths_df)

430154

In [22]:
image_paths_df_total = pd.concat([image_paths_df,image_paths_df_not_inj],axis=0, ignore_index=True)

In [23]:
#Separate the serial_id with more than 50 images, less than 50 images and exactly 50 images
pidf = pd.DataFrame()
lidf = pd.DataFrame()
midf = pd.DataFrame()
num_slices = 50
for serial, group in image_paths_df_total.groupby('serial_id'):
    if len(group) == num_slices:
        group_sorted = group.sort_values('image')
        pidf = pd.concat([pidf,group_sorted],axis=0,ignore_index=True)
    elif len(group) > num_slices:
        group_sorted = group.sort_values('image')
        midf = pd.concat([midf,group_sorted],axis=0,ignore_index=True)
    else:
        group_sorted = group.sort_values('image')
        lidf = pd.concat([lidf,group_sorted],axis=0,ignore_index=True)

In [24]:
new_more_df = pd.DataFrame()  
for serial_id, group in midf.groupby('serial_id'):
    total_rows = group.shape[0]
    step_size = max(total_rows // num_slices, 1)  
    selected_rows = group.iloc[::step_size][:num_slices].reset_index(drop=True)
    new_more_df = pd.concat([new_more_df, selected_rows], ignore_index=True)

In [25]:
new_less_df = pd.DataFrame()
for serial, group in lidf.groupby('serial_id'):
    length = len(group)
    intergers = num_slices//length
    rest = num_slices%length
    group_df = pd.concat([group] * intergers, axis=0)
    if rest >0:
        step_size = max(length // rest, 1)
        selected_rows = group.iloc[::step_size][:rest]
        inter_df = pd.concat([group_df,selected_rows], axis=0)
    new_less_df = pd.concat([new_less_df,inter_df], axis=0)
    new_less_df = new_less_df.sort_index()

In [35]:
df_inj_ninj = pd.concat([df_injuried,df_not_injuried_200],axis=0,ignore_index=True)

In [36]:
total_dfs = pd.concat([pidf,new_less_df,new_more_df],axis=0,ignore_index=True)
df_total = pd.merge(total_dfs,df_inj_ninj, on='patient_id')

In [41]:
df_serials = pd.concat([df_serials_inj,df_serials_n_inj],axis=0,ignore_index=True)

In [42]:
from sklearn.model_selection import train_test_split
df_s = pd.merge(df_serials, df_injuried,on = 'patient_id')
X = df_s['serial_id']
y = df_s[['bowel_healthy',
          'bowel_injury', 
          'extravasation_healthy', 
          'extravasation_injury',
          #'kidney_healthy', 
          #'kidney_low', 
          #'kidney_high', 
          'liver_healthy',
          #'liver_low', 
          #'liver_high', 
          'spleen_healthy',
          'spleen_low',
          'spleen_high'
         ]]
X_train, X_test, y_train, y_test = train_test_split(
    X,y , test_size=0.2, stratify=y, random_state=101
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train ,y_train , test_size=0.1, stratify=y_train, random_state=101
)

In [43]:
train = df_total[df_total['serial_id'].isin(X_train)]
test = df_total[df_total['serial_id'].isin(X_test)]
val = df_total[df_total['serial_id'].isin(X_val)]

In [44]:
def create_sequences_with_step_image(data: pd.DataFrame, seq_len, step):
    sequences = []
    data_size = len(data)

    for i in tqdm(range(0, data_size - seq_len + 1, step)):
        feature = data.iloc[i: i + seq_len,3]
        label_position = (i+seq_len-1)
        label = data.iloc[label_position,5:-1]
        
        sequences.append([feature,label])
        
    return sequences

seq_len = num_slices

In [45]:
train_s = create_sequences_with_step_image(train,seq_len,seq_len)
test_s = create_sequences_with_step_image(test,seq_len,seq_len)
val_s = create_sequences_with_step_image(val,seq_len,seq_len)

100%|██████████| 103/103 [00:00<00:00, 4221.11it/s]


In [11]:
num_slices=50
test = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv')
test_pat_id = test['patient_id'].unique()

# Define the root directory where the patient data is located
root_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'

# Define the list of patient IDs you want to use (replace with your selected IDs)
selected_patient_ids = test_pat_id 

# Initialize empty lists to store data
patient_ids = []
image_paths = []

# Traverse the directories and collect the data
for patient_id in selected_patient_ids:
    patient_dir = os.path.join(root_dir, str(patient_id))
    if os.path.exists(patient_dir):
        for serial_id in os.listdir(patient_dir):
            serial_dir = os.path.join(patient_dir, serial_id)
            if os.path.isdir(serial_dir):
                # List all .dcm files in the serial_id folder
                image_files = [filename for filename in os.listdir(serial_dir) if filename.endswith('.dcm')]
                
                # Calculate the step size for evenly spaced selection
                total_images = len(image_files)
                if total_images <= num_slices:
                    step_size = 1  # If fewer than num_slices images, select all of them
                else:
                    step_size = total_images // num_slices
                
                # Select images in an evenly spaced manner
                selected_images = [image_files[i] for i in range(0, total_images, step_size)][:num_slices]
                
                for filename in selected_images:
                    image_path = os.path.join(serial_dir, filename)
                    patient_ids.append(patient_id)
                    image_paths.append(image_path)

# Create a DataFrame from the collected data
df_test = {'patient_id': patient_ids, 'image_path': image_paths}
df_test = pd.DataFrame(df_test)
df_test_num_slices = pd.DataFrame()
for i in df_test.index:
    df= pd.concat([df_test.iloc[i:i+1,:]]*num_slices,axis=0, ignore_index=True)
    df_test_num_slices = pd.concat([df_test_num_slices,df],axis=0, ignore_index=True)
def create_sequences_with_step_image_inf(data: pd.DataFrame, seq_len, step):
    sequences = []
    data_size = len(data)

    for i in tqdm(range(0, data_size - seq_len + 1, step)):
        feature = data.iloc[i: i + seq_len,1]
        sequences.append([feature])
    return sequences

seq_len = num_slices
test_seq = create_sequences_with_step_image_inf(df_test_num_slices,seq_len,seq_len)

100%|██████████| 3/3 [00:00<00:00, 6403.52it/s]


In [ ]:
np.save('train_s.npy', train_s)
np.save('test_s.npy', train_s)
np.save('val_s.npy', train_s)
np.save('test_seq.npy', test_seq)

In [ ]:
train_s = np.load('/kaggle/input/csvfile/train_s.npy')
test_s = 

In [12]:
class SeqToImageDataset3D(Dataset):

    def __init__(self,sequences,
                ):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):   
        sequence, labels = self.sequences[idx]
        imgs = []
        mean = [0.485]*50
        std = [0.225]*50
        for image_path in sequence:
            path = image_path
            ds = pydicom.dcmread(path)
            image = ds.pixel_array
            image = Image.fromarray(image).convert('L')
            image = image.resize((224,224))
            array = np.array(image)
            imgs.append(array)
        imgs = np.array(imgs)
        t = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
        tspd = imgs.transpose(1,2,0)
        imgs = t(tspd)
        return {'sequence': imgs, 'labels': torch.tensor(labels, dtype=torch.float32)}

In [13]:
class SeqToImageDataset3DINF(Dataset):

    def __init__(self,sequences,
                ):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):   
        sequence = self.sequences[idx]
        imgs = []
        mean = [0.485]*50
        std = [0.225]*50
        for image_path in sequence[0]:
            path = image_path
            ds = pydicom.dcmread(path)
            image = ds.pixel_array
            image = Image.fromarray(image).convert('L')
            image = image.resize((224,224))
            array = np.array(image)
            imgs.append(array)
        imgs = np.array(imgs)
        t = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
        tspd = imgs.transpose(1,2,0)
        imgs = t(tspd)
        return imgs

In [14]:
class CustomDataModule3D(L.LightningDataModule):
    def __init__(self, train_dataframe, val_dataframe, test_dataframe, 
                 pred_dataframe, 
                 batch_size=config.BATCH_SIZE):
        super().__init__()
        self.train_dataframe = train_dataframe
        self.val_dataframe = val_dataframe
        self.test_dataframe = test_dataframe
        self.pred_dataframe = pred_dataframe

        self.batch_size = batch_size
        
        random.seed(config.SEED)
        np.random.seed(config.SEED)
        torch.manual_seed(config.SEED)

    def setup(self, stage=None):
        self.train_dataset = SeqToImageDataset3D(self.train_dataframe)
        self.val_dataset = SeqToImageDataset3D(self.val_dataframe)
        self.test_dataset = SeqToImageDataset3D(self.test_dataframe)
        self.pred_dataset = SeqToImageDataset3DINF(self.pred_dataframe)

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=1
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.val_dataset,
            batch_size=len(self.val_dataframe)//5,
            shuffle=False,
            num_workers=1
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_dataset,
            batch_size=len(self.test_dataframe)//5,
            shuffle=False,
            num_workers=1
        )

    def predict_dataloader(self):
        return DataLoader(
            dataset=self.pred_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=1
        )

In [15]:
class ResNetImage(nn.Module):
    
    def __init__(self):
        super(ResNetImage, self).__init__()
        
        self.features = torchvision.models.resnet18(weights='DEFAULT')
        
        self.features.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=num_slices, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        
        self.features.fc = nn.Linear(in_features=512, out_features=1000, bias = True)
        
        self.layersB = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=2, bias=True),
        )
        
        self.layersE = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=2, bias=True),
        )
        
        
        self.layersK = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=3, bias=True),
        )
        
        
        self.layersL = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=3, bias=True),
        )
  
        
        self.layersS = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=3, bias=True),
        )
    
    def forward(self, x):       
        out = self.features(x)  # Pass each image through the feature extractor
        
        # Perform classification for each task
        bowel_logits = self.layersB(out)
        extravasation_logits = self.layersE(out)
        kidney_logits = self.layersK(out)
        liver_logits = self.layersL(out)
        spleen_logits = self.layersS(out)
        
        return {
            'bowel': bowel_logits,
            'extravasation': extravasation_logits,
            'kidney': kidney_logits,
            'liver': liver_logits,
            'spleen': spleen_logits
        }

In [16]:
device = 'cuda'
#Promissor e funcionando
class LightM(L.LightningModule):
    def __init__(self, model, lr=config.LR):
        super(LightM, self).__init__()
        self.lr = lr
        self.model = model
        
        self.loss_fn1 = nn.BCEWithLogitsLoss()
        self.loss_fn2 = nn.BCEWithLogitsLoss()
        self.loss_fn3 = nn.CrossEntropyLoss()
        self.loss_fn4 = nn.CrossEntropyLoss()
        self.loss_fn5 = nn.CrossEntropyLoss()

    def forward(self, x):

        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])
        
        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('train_bowel_loss', bowel_loss)
        self.log('train_extravasation_loss', extravasation_loss)
        self.log('train_kidney_loss', kidney_loss)
        self.log('train_liver_loss', liver_loss)
        self.log('train_spleen_loss',spleen_loss)
        self.log('train_loss',total_loss, prog_bar=True, logger=True)

        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('train_bowel_acc', accuracy1)
        self.log('train_extravasation_acc', accuracy2)
        self.log('train_kidney_acc', accuracy3)
        self.log('train_liver_acc', accuracy4)
        self.log('train_spleen_acc',accuracy5)
        self.log('train_acc',acc, prog_bar=True, logger=True)

        return total_loss
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])


        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('val_bowel_loss', bowel_loss)
        self.log('val_extravasation_loss', extravasation_loss)
        self.log('val_kidney_loss', kidney_loss)
        self.log('val_liver_loss', liver_loss)
        self.log('val_spleen_loss',spleen_loss)
        self.log('val_loss',total_loss, prog_bar=True, logger=True)
        
        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('val_bowel_acc', accuracy1)
        self.log('val_extravasation_acc', accuracy2)
        self.log('val_kidney_acc', accuracy3)
        self.log('val_liver_acc', accuracy4)
        self.log('val_spleen_acc',accuracy5)
        self.log('val_acc',acc, prog_bar=True, logger=True)
        
        return total_loss
    
    def test_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])


        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('test_bowel_loss', bowel_loss)
        self.log('test_extravasation_loss', extravasation_loss)
        self.log('test_kidney_loss', kidney_loss)
        self.log('test_liver_loss', liver_loss)
        self.log('test_spleen_loss',spleen_loss)
        self.log('test_loss',total_loss, prog_bar=True, logger=True)
        
        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('test_bowel_acc', accuracy1)
        self.log('test_extravasation_acc', accuracy2)
        self.log('test_kidney_acc', accuracy3)
        self.log('test_liver_acc', accuracy4)
        self.log('test_spleen_acc',accuracy5)
        self.log('test_acc',acc, prog_bar=True, logger=True)
        
        return total_loss
    
    def prediction_step(self, batch, batch_idx):
        inputs = batch

        logits = self(inputs)

        return logits

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=57, eta_min = 0.0005) # New!

        return {
            "optimizer": opt,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "train_loss",
                "interval": "step",
                "frequency": 1
            },
        }

In [19]:
#train_s = np.load('/kaggle/input/csvfile/train_s.npy', allow_pickle=True)
#test_s = np.load('/kaggle/input/csvfile/test_s.npy', allow_pickle=True)
#test_seq = np.load('/kaggle/input/csvfile/test_seq.npy', allow_pickle=True)
#val_s = np.load('/kaggle/input/csvfile/val_s.npy', allow_pickle=True)

In [20]:
module = ResNetImage()
model = LightM(model=module, lr=config.LR)

dm = CustomDataModule3D(train_s,
                         val_s,
                         test_s,
                         test_seq,
                         batch_size=config.BATCH_SIZE,
                        )

logger_v2 = TensorBoardLogger(save_dir="/kaggle/working/", name="logs_v2")
callback_v2 = [ModelCheckpoint(save_top_k=1,
                             verbose=True,
                             monitor='val_loss',
                             save_last = True,
                             mode='min',
                             filename='best_model',
                             dirpath = logger_v2.log_dir
                              ),
               EarlyStopping(monitor='val_loss',
                             patience=3,
                             mode='min',
                            )
              ]

trainer = L.Trainer(
    max_epochs=config.MAX_EPOCHS,
    callbacks = callback_v2,
    accelerator="gpu",
    logger=logger_v2,
    deterministic=True,
    precision='16-mixed',
    detect_anomaly=True,
    log_every_n_steps=1
)

In [ ]:
trainer.fit(model,datamodule=dm)

In [ ]:
trainer.test(datamodule=dm, ckpt_path='best', verbose=True)